In [2]:
import pandas as pd
import os
import numpy as np
import re

# How datasets are joined

### Movie dataset on Character dataset
We can use freebase_movie_id

### Character dataset on Oscar dataset
Oscar dataset does not have freebase_movie_id or freebase_actor_id. We instead use parsed_actor_name and movie_identifier. parsed_actor_name will be unique for each movie as we drop actors if they share parsed_actor_name with another actor in the same movie. movie_identifier is a combination of parsed_movie_name and release_year. This is unique as we drop movies that share movie identifier.

# Preparing Movie dataset

In [164]:
movie_df = pd.read_csv('data/movie.metadata.tsv', sep='\t', names=['wiki_movie_id', 'freebase_movie_id', 'title', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres'], index_col='freebase_movie_id')
movie_df = movie_df.reset_index()

In [84]:
def parse_string(s):
    try:
        s = s.lower().strip()
        s  = re.sub('[^a-zA-Z0-9 -]', '', s)
        if len(s) == 0:
            return None
        return s
    except:
        return None 

In [166]:
movie_df['parsed_movie_name'] = movie_df['title'].apply(parse_string)
movie_df['release_year'] = movie_df['release_date'].apply(lambda x: pd.to_datetime(x, format='mixed', errors='coerce').year)

In [170]:
movie_df['release_year'] = movie_df['release_year'].fillna(0).astype(int)

In [171]:
#Dropped movies without parsed_movie_name
movie_df = movie_df[movie_df['parsed_movie_name'].notna()]
#Create identifier
movie_df['movie_identifier'] = movie_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['release_year']), axis=1)
movie_df = movie_df.drop(['parsed_movie_name'], axis=1)

/var/folders/q8/ngmz2fx96cg6d3rs8tldz4r00000gn/T/ipykernel_41489/2069076716.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df['movie_identifier'] = movie_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['release_year']), axis=1)


In [172]:
def extract_column(s):
    s = str(s)
    return re.findall('\"([^\/:][\w\s]+)"', s)

<>:3: SyntaxWarning: invalid escape sequence '\/'
<>:3: SyntaxWarning: invalid escape sequence '\/'
/var/folders/q8/ngmz2fx96cg6d3rs8tldz4r00000gn/T/ipykernel_41489/2187841618.py:3: SyntaxWarning: invalid escape sequence '\/'
  return re.findall('\"([^\/:][\w\s]+)"', s)


In [173]:
# Extract data from columns
movie_df['countries'] = movie_df['countries'].apply(extract_column)
movie_df['languages'] = movie_df['languages'].apply(extract_column)
movie_df['genres'] = movie_df['genres'].apply(extract_column)

In [174]:
movie_df['country'] = movie_df['countries'].map(lambda x: None if len(x) == 0 else x[0])
movie_df['language'] = movie_df['languages'].map(lambda x: 'English Language' if len(x) == 0 else x[0])
movie_df = movie_df.drop(['countries', 'languages'], axis=1)

In [175]:
# Remove movies with duplicate movie_identifiers
temp = movie_df.groupby('movie_identifier').agg(count = ('movie_identifier', 'size'))
print('Number of movies with duplicate movie identifier:', temp[temp['count'] > 1]['count'].sum())
movie_df = movie_df.merge(temp[temp['count'] == 1], on='movie_identifier', how='inner')

movie_df = movie_df.drop('count', axis=1)

Number of movies with duplicate movie identifier: 379


# Preparing Character dataset

In [176]:
# Read character
col_names = ["wiki_movie_id", "freebase_movie_id", "movie_release_date", "character_name", "actor_date_of_birth", "Actor gender", "Actor height (in meters)", "Actor ethnicity (Freebase ID)", "Actor name", "Actor age at movie release", "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
character_df = pd.read_csv('data/character.metadata.tsv', sep='\t', names=col_names, index_col="Freebase character ID")

In [177]:
# Remove characters without actors
character_df = character_df[character_df['Freebase actor ID'].notna()]

# Allow only one character per actor per movie.
# Motivation: When we look at if the actor won a Oscar we don't care about what role it won it as. 
character_df = character_df.groupby(['freebase_movie_id', 'Freebase actor ID']).first().reset_index()

character_df['parsed_actor_name'] = character_df['Actor name'].apply(parse_string)

# How many cases where the actors of the same name stars in the same movie or
temp = character_df.groupby(['freebase_movie_id', 'parsed_actor_name']).agg(count = ('parsed_actor_name', 'size')).reset_index()
movies_with_actors_of_same_name_df = temp[temp['count'] > 1]
print('Number of movies with actors of the same name:', movies_with_actors_of_same_name_df.shape[0])

# Only 313 we decide to drop the cases.
# TODO: Check if any of the 311 actors won Oscars
character_df = character_df.merge(temp, on=['freebase_movie_id', 'parsed_actor_name'], how='inner')
dropped_character_df = character_df[character_df['count'] == 1]

temp = dropped_character_df.groupby(['freebase_movie_id', 'parsed_actor_name']).agg(count = ('parsed_actor_name', 'size')).reset_index()

assert temp[temp['count'] > 1].empty

character_df = dropped_character_df
character_df = character_df.drop('count', axis=1)

Number of movies with actors of the same name: 314


# Merge Movie dataset on character dataset

In [178]:
movie_character_df = movie_df.merge(character_df, on='freebase_movie_id', how='inner')
movie_character_df = movie_character_df[[
    'title',
    'release_date',
    'box_office_revenue',
    'runtime',
    'language',
    'country',
    'genres',
    'movie_identifier',
    'Freebase actor ID',
    'Actor gender',
    'Actor height (in meters)',
    'Actor ethnicity (Freebase ID)',
    'Actor name',
    'Actor age at movie release',
    'parsed_actor_name'
]]
movie_character_df['actor_identifier'] = movie_character_df['Freebase actor ID']
movie_character_df = movie_character_df.drop('Freebase actor ID', axis=1)

In [180]:
# Create identifier used for joining on Oscar dataset
movie_character_df['identifier'] = movie_character_df.apply(lambda x: x['movie_identifier'] + '_' + x['parsed_actor_name'], axis=1)
assert movie_character_df['identifier'].is_unique

# Preparing Oscar dataset

In [181]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')

# Removes weird Oscar nomination
oscar_df = oscar_df[oscar_df['film'].notna() & oscar_df['name'].notna()]

In [182]:
# Filtering relevant Oscar catagories
# TODO: explain why we drop certain categories.
oscar_df = oscar_df[oscar_df['category'].str.contains('ACTOR') | oscar_df['category'].str.contains('ACTRESS')]

In [183]:
# Create identifier 
oscar_df['parsed_movie_name'] = oscar_df['film'].apply(parse_string)
oscar_df['parsed_actor_name'] = oscar_df['name'].apply(parse_string)
oscar_df['identifier'] = oscar_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['year_film']) + '_' + x['parsed_actor_name'], axis=1)

In [184]:
# If rows share identifier it must mean the same actor got nominated for the 
# same film in multiple categories. We only count one.
oscar_df = oscar_df.groupby(['identifier']).first().reset_index()
assert oscar_df['identifier'].is_unique

In [191]:
# Select relevant columns
oscar_df = oscar_df[[
    'identifier',
    'category',
    'winner'
]]

# Merge movie_character dataset to Oscar
The join is identifier which consists of parsed_movie_name, release_year and parsed_actor_name

In [249]:
movie_character_oscar_df = movie_character_df.merge(oscar_df, how='left', on='identifier')

In [250]:
# Create column to determine if actor was nominated for Oscar 
movie_character_oscar_df['oscar_nominated'] = movie_character_oscar_df['category'].notna()

## Stats about dataset

In [251]:
nominated_df = movie_character_oscar_df[movie_character_oscar_df['oscar_nominated'] == True]
print('Number of different Oscar nominated movies in dataset:', 
      nominated_df['movie_identifier'].unique().shape[0])
print('Number of different movies in dataset:', 
      movie_character_oscar_df['movie_identifier'].unique().shape[0])
print('Number of different Oscar nominated actors in dataset:', 
      nominated_df['actor_identifier'].unique().shape[0])
print('Number of different actors in dataset:', 
      movie_character_oscar_df['actor_identifier'].unique().shape[0])
print('Number of Oscar nominated rows:', nominated_df.shape[0])

# More actors than movies. Why: Actors can win more oscars

Number of different Oscar nominated movies in dataset: 952
Number of different movies in dataset: 63968
Number of different Oscar nominated actors in dataset: 801
Number of different actors in dataset: 134907
Number of Oscar nominated rows: 1443


# Prepare IMDB rating dataset 

In [252]:
#Read data
titlebasics_df = pd.read_csv('data/title.basics.tsv', sep='\t', quoting=3)
titleratings_df = pd.read_csv('data/title.ratings.tsv', sep='\t')

In [204]:
#Merge dataframes containing ratings and title names
name_rating_df = titlebasics_df.merge(titleratings_df, how='inner', on=['tconst', 'tconst'])
#Consider only movies
name_rating_df = name_rating_df[(name_rating_df["titleType"] == 'movie')]
#Get relevant columns
name_rating_df = name_rating_df[["primaryTitle", "startYear", "averageRating", "numVotes"]]

In [205]:
#Change the year data in IMDB dataset to float for merging to work
#\\N is a special value used for missing, replace with NaN so it can't be used for merging
name_rating_df["startYear"] = name_rating_df["startYear"].replace('\\N', np.nan)
name_rating_df["startYear"] = name_rating_df["startYear"].astype(float)
#Rename columns for merge
name_rating_df.rename(columns={'primaryTitle': 'title', 'startYear' : 'year', 'titleType': 'type'}, inplace=True)

In [206]:
#Have to take care of duplicate entries for movies in the same year
#ASSUME they are the same movie and aggregate the scores: sum up the numVotes and calculate the
#new average rating taking into account the number of votes

#The ratings need to be weighted to account for the number of votes
name_rating_df["RatingWeight"] = name_rating_df['averageRating'] * name_rating_df['numVotes']
# Group by title and year for duplicates, sum the number of votes and the weighted ratings
name_rating_agg_df = name_rating_df.groupby(['title','year']).agg(
    numVotes=('numVotes', 'sum'),
    RatingWeight = ('RatingWeight','sum'), 
).reset_index()

#Undo the previous weighing, dividing by number of all votes
name_rating_agg_df['averageRating'] = name_rating_agg_df['RatingWeight'] / name_rating_agg_df['numVotes']
#All ratings have 1 space after comma
name_rating_agg_df['averageRating'] = name_rating_agg_df['averageRating'].round(1)

#Drop the temporary weighted ratings
name_rating_agg_df.drop(columns='RatingWeight', inplace=True)

In [207]:
#Get the year of release of the movies from the mixed formatting
dates_as_year = pd.to_datetime(movie_character_oscar_df["release_date"],format='mixed',errors='coerce').dt.year

#Copy of the dataset with the added year column
movie_character_oscar_df['year'] = dates_as_year 

rows_before_ratings = movie_character_oscar_df.shape[0]

# Join movie_character_oscar on IMDB dataset

In [210]:
name_rating_agg_df

,title,year,numVotes,averageRating
0,!Women Art Revolution,2010.0,261,6.9
1,"""Flip Flop Flippin': One man's search for char...",2014.0,25,6.1
2,"""Giliap""",1975.0,969,6.3
3,"""Has Anyone Seen Viljo?""",1988.0,335,4.4
4,#1 Serial Killer,2013.0,72,5.6
...,...,...,...,...
317862,Üçünüzü de mihlarim,1965.0,44,5.4
317863,Üçüzler,1993.0,7,3.6
317864,Þegiðu og syntu,2020.0,16,9.4
317865,à propos: philosophie,2016.0,8,8.5


In [216]:
# Create identifier
imdb_df = name_rating_agg_df.copy(deep=True)
imdb_df['parsed_movie_name'] = imdb_df.apply(lambda x: parse_string(x['title']), axis=1)
imdb_df['release_year'] = imdb_df['year'].fillna(0).astype(int)
imdb_df = imdb_df[imdb_df['parsed_movie_name'].notna()]
imdb_df['movie_identifier'] = imdb_df.apply(lambda x: x['parsed_movie_name'] + '_' + str(x['release_year']), axis=1)

In [222]:
# Drop ratings of movies that share movie identifier
temp = imdb_df.groupby('movie_identifier').agg(count = ('movie_identifier', 'size')).sort_values('count', ascending=False)
temp = temp[temp['count'] == 1]
imdb_df = imdb_df.merge(temp, on='movie_identifier', how='inner')
imdb_df = imdb_df.drop('count', axis=1)
assert imdb_df['movie_identifier'].is_unique

In [227]:
# Select relevant columns
imdb_df = imdb_df[[
    'averageRating',
    'numVotes',
    'movie_identifier'
]]

In [253]:
movie_character_oscar_rating_df = movie_character_oscar_df.merge(imdb_df, on='movie_identifier', how='left')
movie_character_oscar_rating_df['has_rating'] = movie_character_oscar_rating_df['numVotes'].notna()

In [245]:
movie_character_oscar_rating_df['average_rating'] = movie_character_oscar_rating_df['averageRating']
movie_character_oscar_rating_df['number_of_votes'] = movie_character_oscar_rating_df['numVotes']
movie_character_oscar_rating_df = movie_character_oscar_rating_df.drop(['numVotes', 'averageRating'], axis=1)

In [ ]:
## Stats of dataset
rating_df = movie_character_oscar_rating_df[movie_character_oscar_rating_df['has_rating'] == True]
print('Number of movies with ratings:', rating_df['movie_identifier'].unique().shape[0])
print('Oscar nominated movies with rating:', rating_df[rating_df['oscar_nominated'] == True]['movie_identifier'].shape[0])

# Basically all Oscar nominated movies have rating

Number of movies with ratings: 36757
Oscar nominated movies with rating: 1426


We have three columns for date - `release_date`, `year_ceremony` and `year_film`. `year_film` and `year_ceremony` are from the oscar dataset and are only available for 1827 rows. Additionally, `year_ceremony` is always exactly 1 more than `year_film`. This allows us to conclude that the `year_film` and `year_ceremony` columns are redundant and can be dropped.

For similar reasons, the `name` and `film` columns from the oscar dataset are also redundant.

The `Freebase character/actor map ID` column is also not useful for us as we only care about actors and not the characters played by them in movies. 

From the pairs of columns (`title`,`parsed_movie_name`) and (`Actor_name`, `parsed_actor_name`), we choose to keep the parsed versions. At first glance, it might appear that the parsed columns have no missing values while the original ones do not, but this is simply because the missing values have been converted to 'nan' strings. This is something to be mindful of. TODO we might even want to drop rows with `nan` actors?



In [258]:
movie_character_oscar_rating_df

,title,release_date,box_office_revenue,runtime,language,country,genres,movie_identifier,Actor gender,Actor height (in meters),...,Actor age at movie release,parsed_actor_name,actor_identifier,identifier,category,winner,oscar_nominated,averageRating,numVotes,has_rating
0,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,"[Thriller, Science Fiction, Horror, Adventure,...",ghosts of mars_2001,M,1.727,...,32.0,ice cube,/m/01vw26l,ghosts of mars_2001_ice cube,NaN,NaN,False,4.9,58894.0,True
1,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,"[Thriller, Science Fiction, Horror, Adventure,...",ghosts of mars_2001,F,1.650,...,23.0,clea duvall,/m/01y9xg,ghosts of mars_2001_clea duvall,NaN,NaN,False,4.9,58894.0,True
2,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,"[Thriller, Science Fiction, Horror, Adventure,...",ghosts of mars_2001,M,NaN,...,47.0,robert carradine,/m/026qxz,ghosts of mars_2001_robert carradine,NaN,NaN,False,4.9,58894.0,True
3,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,"[Thriller, Science Fiction, Horror, Adventure,...",ghosts of mars_2001,M,NaN,...,66.0,doug mcgrath,/m/02r5d3j,ghosts of mars_2001_doug mcgrath,NaN,NaN,False,4.9,58894.0,True
4,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,"[Thriller, Science Fiction, Horror, Adventure,...",ghosts of mars_2001,F,1.780,...,27.0,natasha henstridge,/m/0346l4,ghosts of mars_2001_natasha henstridge,NaN,NaN,False,4.9,58894.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443499,Spliced,2002,NaN,86.0,English Language,Canada,"[Thriller, Horror, Slasher, Teen]",spliced_2002,None,NaN,...,19.0,andrea runge,/m/0gbx_rk,spliced_2002_andrea runge,NaN,NaN,False,NaN,NaN,False
443500,Spliced,2002,NaN,86.0,English Language,Canada,"[Thriller, Horror, Slasher, Teen]",spliced_2002,F,NaN,...,NaN,wendy anderson,/m/0gby01h,spliced_2002_wendy anderson,NaN,NaN,False,NaN,NaN,False
443501,Spliced,2002,NaN,86.0,English Language,Canada,"[Thriller, Horror, Slasher, Teen]",spliced_2002,None,NaN,...,NaN,billy morton,/m/0gc4lfm,spliced_2002_billy morton,NaN,NaN,False,NaN,NaN,False
443502,Spliced,2002,NaN,86.0,English Language,Canada,"[Thriller, Horror, Slasher, Teen]",spliced_2002,None,NaN,...,NaN,ariel bastian,/m/0gdkb55,spliced_2002_ariel bastian,NaN,NaN,False,NaN,NaN,False


## Output data

In [257]:
# Finish processed data
path = 'cache/data.csv'
os.makedirs('cache', exist_ok=True)
movie_character_oscar_rating_df.to_csv(path)
print('Processing done')

Processing done
